In [1]:
# Preamble
from astropy.table import Table, column, vstack
import numpy as np
import os
import pandas as pd

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

import sys
sys.path.append('../astro_codes/')

In [2]:
gz2_data = Table.read('../../fits/full_sample_debiased_w_low_z_mod.fits')

In [73]:
mass_masks_matched = Table.read('../fits/mass_masks_matched.fits')
mendel_matched = Table.read('../fits/mendel_matched.fits')
gz2_stats = Table.read('../fits/gz2_statistics.fits')
ml_table_matched = Table.read('../fits/ml_table_matched.fits')
sparcfire_stats = Table.read('../fits/galaxy_level_matched.fits')

In [4]:
sample_ok = ml_table_matched['vmax_ok']
sparcfire_ok = sparcfire_stats['N_arcs'] >= 1 

In [156]:
from tabulate import tabulate

category_list = [r'spiral (all)',
                 r'$m=1$ (all)',
                 r'$m=2$ (all)',
                 r'$m=3$ (all)',
                 r'$m=4$ (all)',
                 r'$m=5+$ (all)',
                 r'spiral ($p_\mathrm{bar} \leq 0.2$)',
                 r'$m=1$ ($p_\mathrm{bar} \leq 0.2$)',
                 r'$m=2$ ($p_\mathrm{bar} \leq 0.2$)',
                 r'$m=3$ ($p_\mathrm{bar} \leq 0.2$)',
                 r'$m=4$ ($p_\mathrm{bar} \leq 0.2$)*',
                 r'$m=5+$ ($p_\mathrm{bar} \leq 0.2$)',
                 r'spiral ($0.2 < p_\mathrm{bar} \leq 0.5$)',
                 r'$m=1$ ($0.2 < p_\mathrm{bar} \leq 0.5$)',
                 r'$m=2$ ($0.2 < p_\mathrm{bar} \leq 0.5$)',
                 r'$m=3$ ($0.2 < p_\mathrm{bar} \leq 0.5$)',
                 r'$m=4$ ($0.2 < p_\mathrm{bar} \leq 0.5$)',
                 r'$m=5+$ ($0.2 < p_\mathrm{bar} \leq 0.5$)',
                 r'spiral ($p_\mathrm{bar} > 0.5$)',
                 r'$m=1$ ($p_\mathrm{bar} > 0.5$)',
                 r'$m=2$ ($p_\mathrm{bar} > 0.5$)',
                 r'$m=3$ ($p_\mathrm{bar} > 0.5$)',
                 r'$m=4$ ($p_\mathrm{bar} > 0.5$)',
                 r'$m=5+$ ($p_\mathrm{bar} > 0.5$)']


gz2_stats['p_bar'] = gz2_data['t03_bar_a06_bar_debiased_rh']
bar_masks = [gz2_stats['p_bar'] <= 0.2,
             (gz2_stats['p_bar'] > 0.2) & (gz2_stats['p_bar'] <= 0.5),
             gz2_stats['p_bar'] > 0.5]

names_ = mass_masks_matched.colnames[3:]

masks = []
# -------------------------------------------------------
masks.append(np.any([mass_masks_matched[c] == 1
                     for c in names_],axis=0))
for m in range(5):
    masks.append(np.any([mass_masks_matched[names_[m_]] == 1
                        for m_ in [m,m+5,m+10]],axis=0))
# -------------------------------------------------------
masks.append(np.any([mass_masks_matched[c] == 1
                     for c in names_[:5]],axis=0))
for m in range(5):
    masks.append(mass_masks_matched[names_[m]] == 1)
# -------------------------------------------------------
masks.append(np.any([mass_masks_matched[c] == 1
                     for c in names_[5:10]],axis=0))
for m in range(5):
    masks.append(mass_masks_matched[names_[m+5]] == 1)
# -------------------------------------------------------
masks.append(np.any([mass_masks_matched[c] == 1
                     for c in names_[10:]],axis=0))
for m in range(5):
    masks.append(mass_masks_matched[names_[m+10]] == 1)
# -------------------------------------------------------


# -------------------------------------------------------
all_masks = []
all_masks.append(sample_ok)
for m in range(5):
    all_masks.append(np.all([sample_ok,gz2_stats['m'] == m+1],axis=0))
# -------------------------------------------------------
bar_ok = np.all([bar_masks[0],sample_ok],axis=0)
all_masks.append(bar_ok)
for m in range(5):
    all_masks.append(np.all([bar_ok,gz2_stats['m'] == m+1],axis=0))
# -------------------------------------------------------
bar_ok = np.all([bar_masks[1],sample_ok],axis=0)
all_masks.append(bar_ok)
for m in range(5):
    all_masks.append(np.all([bar_ok,gz2_stats['m'] == m+1],axis=0))
# -------------------------------------------------------
bar_ok = np.all([bar_masks[2],sample_ok],axis=0)
all_masks.append(bar_ok)
for m in range(5):
    all_masks.append(np.all([bar_ok,gz2_stats['m'] == m+1],axis=0))
# -------------------------------------------------------

In [157]:
def get_strings(masses,in_sparcfire,mask1,mask2):
    N = (mask1.sum(), np.all([mask1,in_sparcfire],axis=0).sum())
    N_string = '{} ({}%)'.format(N[0],np.round(100*N[1]/N[0],decimals=1))
    
    N_matched = (np.all([mask1,mask2],axis=0).sum(), 
                 np.all([mask1,mask2,in_sparcfire],axis=0).sum())
    N_string_matched = '{} ({}%)'.format(N_matched[0],
                np.round(100*N_matched[1]/N_matched[0],decimals=1))
    
    mass_percentiles = np.percentile(masses[mask1],(50,16,84))
    mass_string = '{} ({}, {})'.format(
        *np.round(mass_percentiles,decimals=2))
    mass_percentiles_matched = np.percentile(masses[(mask1) & (mask2)],
                                             (50,16,84))
    mass_string_matched = '{} ({}, {})'.format(
        *np.round(mass_percentiles_matched,decimals=2))
    return N_string, mass_string, N_string_matched, mass_string_matched

In [160]:
table_array = np.full((24,5),'--',dtype='U100')
table_array[:,0] = category_list

for i, (mask, all_mask) in enumerate(zip(masks,all_masks)):

    table_strings = get_strings(mendel_matched['bulge+disc'],
                                sparcfire_ok,all_mask,mask)
    table_array[i,1:] = table_strings
    
headers = [r'subsample',r'$N$ (all)',r'$\log(M_*/M_\odot)$ (all)',
           r'$N$ ($M_*$-matched)',r'$\log(M_*/M_\odot)$ ($M_*$-matched)']


def tabulator(table_array,headers):
    text_table = tabulate(table_array,headers=headers,tablefmt='latex')
    text_table = text_table.replace('textbackslash{}','')
    text_table = text_table.replace('\$','$')
    text_table = text_table.replace('\{','{')
    text_table = text_table.replace('\}','}')
    text_table = text_table.replace('\_','_')
    text_table = text_table.replace('\ensuremath{<}','<')
    text_table = text_table.replace('\ensuremath{>}','>')
    text_table = text_table.replace('\\%','\%')
    return text_table

text_table = tabulator(table_array,headers)

In [161]:
print(text_table)

\begin{tabular}{lllll}
\hline
 subsample                                & $N$ (all)    & $\log(M_*/M_\odot)$ (all)   & $N$ ($M_*$-matched)   & $\log(M_*/M_\odot)$ ($M_*$-matched)   \\
\hline
 spiral (all)                             & 6222 (48.7\%) & 10.27 (9.89, 10.64)         & 4908 (48.2\%)          & 10.26 (9.92, 10.58)                   \\
 $m=1$ (all)                              & 243 (25.9\%)  & 10.18 (9.78, 10.57)         & 151 (20.5\%)           & 10.25 (9.92, 10.56)                   \\
 $m=2$ (all)                              & 4014 (46.6\%) & 10.26 (9.88, 10.63)         & 3208 (45.6\%)          & 10.26 (9.93, 10.58)                   \\
 $m=3$ (all)                              & 1108 (56.9\%) & 10.32 (9.93, 10.66)         & 876 (57.5\%)           & 10.25 (9.91, 10.56)                   \\
 $m=4$ (all)                              & 405 (54.6\%)  & 10.31 (9.93, 10.67)         & 337 (55.5\%)           & 10.26 (9.92, 10.56)                   \\
 $m=5+$ (all)                

In [125]:
bar_masks[0].sum()

150687